In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../src')

In [5]:
from d00_utils.db_utils import dbReadWriteViews
io_views = dbReadWriteViews()

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [6]:
instances_w_labels_test_downsampleby5_df = io_views.get_table('instances_w_labels_test_downsampleby5')
instances_w_labels_test_downsampleby5_df = instances_w_labels_test_downsampleby5_df[(instances_w_labels_test_downsampleby5_df['view']=='a2c') | (instances_w_labels_test_downsampleby5_df['view']=='a4c')]
instanceidks = instances_w_labels_test_downsampleby5_df['instanceidk'].unique()
print(len(instanceidks))

522


In [7]:
from d05_measurement.write_groundtruth import write_groundtruth
write_groundtruth(instanceidks)

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
